## 한국인의 영어 발화 데이터로 finetuning한  모델의 성능을 파악하기 위한 간단한 테스트 코드  
원하는 음성파일을 불러와서 직접 테스트 가능

In [ ]:
!pip install transformers
!pip install wave

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Load data

In [2]:
!git clone https://github.com/pongjin/sample_wavs.git

Cloning into 'sample_wavs'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 1.29 MiB | 5.67 MiB/s, done.


# 1. Predict 50sec native speech

## change 8hz to 16hz

In [ ]:
import wave
import torch
import scipy.signal as signal
import numpy as np

# WAV 파일 경로 지정
input_file = '/content/sample_wavs/male.wav'

# 입력 WAV 파일 열기
with wave.open(input_file, "rb") as wav_in:
    # 입력 WAV 파일의 샘플 속도 확인
    input_sample_rate = wav_in.getframerate()
    # 입력 WAV 파일의 오디오 데이터 읽기
    audio_data = wav_in.readframes(-1)
    audio_data = np.frombuffer(audio_data, dtype=np.int16)

# 입력 WAV 파일의 샘플 속도와 목표 샘플 속도 비율 계산
rate_ratio = 16000 / input_sample_rate

# 샘플 속도 변경
resampled_audio = signal.resample(audio_data, int(len(audio_data) * rate_ratio))

# 새로운 WAV 파일 생성
with wave.open(input_file, "wb") as wav_out:
    # 출력 WAV 파일 설정
    wav_out.setnchannels(1)  # 모노 오디오
    wav_out.setsampwidth(2)  # 16비트
    wav_out.setframerate(16000)  # 목표 샘플 속도

    # 오디오 데이터를 WAV 파일에 쓰기
    wav_out.writeframes(resampled_audio.astype(np.int16).tobytes())

## model input
1분을 한꺼번에 넣으면 메모리 부족으로 종료됨.  
따라서 10초씩 끊어서 진행( 추후 iterator로 메모리 사용 조절해야될듯..?)

In [ ]:
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

# 사전 학습된 모델과 토크나이저 경로
model_name = "pongjin/en_with_korean_w2v_model"
tokenizer_name = "pongjin/en_with_korean_w2v_model"

# 모델과 토크나이저 로드
model = Wav2Vec2ForCTC.from_pretrained(model_name)
tokenizer = Wav2Vec2Tokenizer.from_pretrained(tokenizer_name)

In [ ]:
# 앞서 16khz로 변경된 WAV 파일 로드
waveform, sample_rate = torchaudio.load(input_file)

# 잘라낼 시간 간격 설정 (초 단위)
interval = 10

# 잘라낸 오디오 데이터 저장할 리스트
sliced_audio_data = []

# 오디오 데이터를 interval 간격으로 자르기
start_time = 0
end_time = interval * sample_rate
while end_time <= waveform.size(1):
    sliced_audio_data.append(waveform[:, start_time:end_time])
    start_time += interval * sample_rate
    end_time += interval * sample_rate

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## predict

In [ ]:
total = []
# 잘라진 오디오 데이터를 모델의 입력값으로 사용
for sliced in sliced_audio_data:

    # 토큰화 및 인코딩
    input_values = tokenizer(sliced, return_tensors="pt").input_values
    input_values = torch.tensor(input_values[0])
    # 모델에 입력 텐서 전달하여 예측
    outputs = model(input_values)

    # CTC 디코딩을 통해 최종 예측 텍스트 얻기
    predicted_ids = outputs.logits.argmax(dim=-1)
    predicted_text = tokenizer.batch_decode(predicted_ids)[0]
    total.append(predicted_text)

total

<ipython-input-15-a8e6d4fa407a>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(input_values[0])


["butw what if somebody dewcides to brak it ecarful that youkeepw atat curverigww but louk for wplacs to save mony maybew itw's wtakwing longerto get thing wswcward away than b",
 'ankers expecatiring the wifeaor one company mawen her taxaated ritirment andcount bostaafaabut i aiqaeta new saut seaving rags ar hurly t',
 'uston to nwgod bons what a descusion wcan insou when th tiple of this tipe o song is in cwuestwionthors no dinw or waknw orgassing d typrwat may be person',
 's own bak whaclayse lather hard place work on a fligh surfice and smoth out he qsimplist kind of seprit systom usis iq single subconqtainedq un',
 'wtothe old shop ate holwwww a god macanic is wusuly a badb so figers would go harw in later years some make beautiful chairs w cabnetsw ches d']

In [ ]:
# real sound
from IPython.display import Audio
Audio(filename='/content/sample_wavs/male.wav')

Output hidden; open in https://colab.research.google.com to view.

## 처음 10초만 transcript를 gpt로 변환한 결과
(change more naturally "문장")  

"But what if somebody decides to break it? Be careful that you keep an eye out for places to save money. Maybe it's taking longer to get things sorted away than expected."

In [ ]:
# 비교
total[0]

"butw what if somebody dewcides to brak it ecarful that youkeepw atat curverigww but louk for wplacs to save mony maybew itw's wtakwing longerto get thing wswcward away than b"

# 성능 저하 원인
1. 입력이 선명하지 않음 : 근데 실사용에서는 이거보다 입력이 흐릴 가능성이 농후함.. 추가적인 전처리 필요?
2. native라서 더 못하는걸수도..? : 한국인의 영어 발화 데이터로 학습했기에 콩글리쉬에 더 적합한 모델이라 이럴수도 있다.

***
# predict 10sec korean speech(already trained data)

In [ ]:
# pcm to wav
import soundfile as sf
import librosa as lr

with open('/content/sample_wavs/SW201105ETRNGSF04623LWJ0073.pcm', 'rb') as tf:
    buf = tf.read()
    buf = buf+b'0' if len(buf)%2 else buf

pcm_data = np.frombuffer(buf, dtype='int16')
wav_data = lr.util.buf_to_float(x=pcm_data, n_bytes=2)
sf.write('/content/sample_wavs/SW201105ETRNGSF04623LWJ0073.wav', wav_data, 16000, format='WAV', endian='LITTLE', subtype='PCM_16')

이미 16hz라 변환 필요 없음

In [ ]:
# 앞서 16khz로 변경된 WAV 파일 로드
waveform, sample_rate = torchaudio.load('/content/sample_wavs/SW201105ETRNGSF04623LWJ0073.wav')

# 잘라낼 시간 간격 설정 (초 단위)
interval = 10

# 잘라낸 오디오 데이터 저장할 리스트
sliced_audio_data = []

# 오디오 데이터를 interval초 간격으로 자르기
start_time = 0
end_time = interval * sample_rate
while end_time <= waveform.size(1):
    sliced_audio_data.append(waveform[:, start_time:end_time])
    start_time += interval * sample_rate
    end_time += interval * sample_rate

In [ ]:
total = []
# 잘라진 오디오 데이터를 모델의 입력값으로 사용
for sliced in sliced_audio_data:

    # 토큰화 및 인코딩
    input_values = tokenizer(sliced, return_tensors="pt").input_values
    input_values = torch.tensor(input_values[0])
    # 모델에 입력 텐서 전달하여 예측 수행
    outputs = model(input_values)

    # CTC 디코딩을 통해 최종 예측 텍스트 얻기
    predicted_ids = outputs.logits.argmax(dim=-1)
    predicted_text = tokenizer.batch_decode(predicted_ids)[0]
    total.append(predicted_text)

<ipython-input-19-a8e6d4fa407a>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(input_values[0])


['i waited an hour for my date but it was obvious that she stood me up']

In [ ]:
total

['i waited an hour for my date but it was obvious that she stood me up']

In [ ]:
from IPython.display import Audio
Audio(filename='/content/sample_wavs/SW201105ETRNGSF04623LWJ0073.wav')